In [24]:
import numpy as np
import pandas as pd
import twython
from twython import TwythonStreamer
import re
from requests_oauthlib import OAuth1
import urllib

import sys
import ast
import json

import findspark
findspark.init('/Users/andrew/Desktop/spark-1.5.1/')

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import requests

import matplotlib.pyplot as plt
import threading
import Queue
import time

%matplotlib inline

In [3]:
APP_KEY = "bv6mnYBiFeEVKvPEZlg"
APP_SECRET = "nQZk9Ca8qqJxc1Za07WyW0VPZ6gtAUSF3oPD5sun0"
OAUTH_TOKEN = "606525030-ilOtJstbRvFCjUNMtOu8DP2HQKGWpQvmUsF6fblE"
OAUTH_TOKEN_SECRET = "xSVE47qVOFxxZm1oqKwL6zwLVMWpzxCUYGmLJ6CVHR0mZ"

In [2]:
findspark.find()

'/Users/andrew/Desktop/spark-1.5.1/'

In [21]:
config_token = OAuth1(APP_KEY,
                      client_secret=APP_SECRET,
                      resource_owner_key=OAUTH_TOKEN,
                      resource_owner_secret=OAUTH_TOKEN_SECRET)

config_url = 'https://stream.twitter.com/1.1/statuses/filter.json'

In [29]:
search_terms = np.loadtxt("search-terms.txt",delimiter="\n",dtype=object)
search_terms = ','.join(search_terms)
search_terms = urllib.urlencode({"track":search_terms}).split("=")[1]
#print search_terms

debate%2Cgopdebate%2Ccnndebate%2Ccnbcdebate%2Cfoxbusinessdebate%2Crepublicandebate%2Cdemocraticdebate%2Ccandidates%2Cclinton%2Cclinton%27s%2Chillary%2Chillary%27s%2Chillaryclinton%2Creadyforhillary%2Csanders%2Csanders%27%2Csanders%27s%2Cbernie%2Cbernie%27s%2Cfeelthebern%2Ctrump%2Ctrump%27s%2Cdonaldtrump%2Cmakeamericagreatagain%2Ccarson%2Ccarson%27s%2Cbencarson%2Cchristie%2Cchristie%27s%2Cchrischristie%2Crand%2Crand%27s%2Crandpaul%2Crubio%2Crubio%27s%2Cmarcorubio%2Cbush%2Cbush%27s%2Cjeb%2Cjeb%27s%2Cjebbush%2Cjebcanfixit%2Ccruz%2Ccruz%27s%2Ctedcruz%2Ckaisch%2Ckaisch%27s%2Cjohnkaisch%2Cfiorina%2Cfiorina%27s%2Ccarlyfiorina%2Cjindal%2Cjindal%27s%2Cbobbyjindal%2Csantorum%2Csantorum%27s%2Cricksantorum%2Chuckabee%2Chuckabee%27s%2Cmikehuckabee


In [31]:
BATCH_INTERVAL = 60  # How frequently to update (seconds)
BLOCKSIZE = 50  # How many tweets per update


def main():
    threads = []
    q = Queue.Queue()
    # Set up spark objects and run
    sc  = SparkContext('local[4]', 'Twitter Stream')
    ssc = StreamingContext(sc, BATCH_INTERVAL)
    threads.append(threading.Thread(target=spark_stream, args=(sc, ssc, q)))
    [t.start() for t in threads]
    
def spark_stream(sc, ssc, q):
    """
    Establish queued spark stream.
    For a **rough** tutorial of what I'm doing here, check this unit test
    https://github.com/databricks/spark-perf/blob/master/pyspark-tests/streaming_tests.py
    * Essentially this establishes an empty RDD object filled with integers [0, BLOCKSIZE).
    * We then set up our DStream object to have the default RDD be our empty RDD.
    * Finally, we transform our DStream by applying a map to each element (remember these
        were integers) and setting the next element to be the next element from the Twitter
        stream.
    * Afterwards we perform the analysis
        1. Convert each string to a literal python object
        2. Filter by keyword association (sentiment analysis)
        3. Convert each object to just the coordinate tuple
    :param sc: SparkContext
    :param ssc: StreamingContext
    """
    # Setup Stream
    rdd = ssc.sparkContext.parallelize([0])
    stream = ssc.queueStream([], default=rdd)

    stream = stream.transform(tfunc)

    '''# Analysis
    coord_stream = stream.map(lambda line: ast.literal_eval(line)) \
                        .filter(filter_posts) \
                        .map(get_coord)

    # Convert to something usable....
    coord_stream.foreachRDD(lambda t, rdd: q.put(rdd.collect()))
    
    '''
    stream.foreachRDD(lambda t, rdd: q.put(rdd.collect()))
    # Run!
    ssc.start()
    ssc.awaitTermination()


def stream_twitter_data():
    """
    Only pull in tweets with location information
    :param response: requests response object
        This is the returned response from the GET request on the twitter endpoint
    """
    data      = [('language', 'en'), ('track', search_terms), ('delimited','length')]
    query_url = config_url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in data])
    response  = requests.get(query_url, auth=config_token, stream=True)
    print(query_url, response) # 200 <OK>
    count = 0
    for line in response.iter_lines():  # Iterate over streaming tweets
        try:
            if count > BLOCKSIZE:
                break
            post     = json.loads(line.decode('utf-8'))
            contents = [post['text'], post['user']['screen_name'], post['user']['location']]
            count   += 1
            yield str(contents)
        except:
            print(line)


def tfunc(t, rdd):
    """
    Transforming function. Converts our blank RDD to something usable
    :param t: datetime
    :param rdd: rdd
        Current rdd we're mapping to
    """
    return rdd.flatMap(lambda x: stream_twitter_data())

if __name__ == '__main__':
    sys.exit(main())